## Calculate Moon position with JPLephem

In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import numpy as np
from astropy.time import Time
from astropy.coordinates import SkyCoord, ICRS, GCRS, EarthLocation, cartesian_to_spherical 
import astropy.units as u

def get_moon(time, location):
    '''
    Use JPLephem to get approximate ICRS coord for moon at ``time``
    '''
    # Get position of moon relative to Earth-moon barycenter with JPLephem
    import jplephem
    from jplephem.spk import SPK
    # Note, this table is only good for years 1550–2650
    kernel = SPK.open('de430.bsp')
    t = Time(time)
    cartesian_position = kernel[3,301].compute(t.jd) # Earth-moon barycenter to moon
    
    # Convert to GCRS coordinates
    d, dec, ra = cartesian_to_spherical(*cartesian_position)
    moon = GCRS(ra, dec, d, obsgeoloc=location)
    return moon

# Set up observer
lat = '00:00:00'
lon = '00:00:00'
location = EarthLocation.from_geodetic(lon, lat, 0)
t = Time('2015-01-01 00:00:00')

# Get the position of the moon for this observer
moon = get_moon(t, location)
print('Moon RA, dec (JPLephem+astropy):', moon.ra.degree, moon.dec.degree)

# Compute alt/az of moon at time with astropy+JPLephem
import sys
sys.path.insert(0, '/Users/bmorris/git/astroplan')
from astroplan import Observer
obs = Observer(location=location, pressure=0)
altaz = obs.altaz(t, moon)
print('Moon alt, az (JPLephem+astropy):', altaz.alt.degree, altaz.az.degree)

# Compute comparisions with PyEphem
import ephem
pyephem_obs = ephem.Observer()
pyephem_obs.lat = lat
pyephem_obs.lon = lon
pyephem_obs.elevation = 0
pyephem_obs.date = t.iso
pyephem_obs.pressure = 0
pyephem_moon = ephem.Moon()
pyephem_moon.compute(pyephem_obs)
print('Moon RA, dec (PyEphem):', np.degrees(pyephem_moon.a_ra), np.degrees(pyephem_moon.a_dec))
print('Moon alt, az (PyEphem):', np.degrees(pyephem_moon.alt), np.degrees(pyephem_moon.az))

Moon RA, dec (JPLephem+astropy): 48.7556183796 15.0494144934
Moon alt, az (JPLephem+astropy): 37.0725678456 289.059811758
Moon RA, dec (PyEphem): 48.7659190423 15.0513836692
Moon alt, az (PyEphem): 36.3122772506 289.064342732
